# Эксперимент с обучением FCRN на малом датасете, random crop

### Датасет

Датасет состоит из 56500 изображений (18800 картинок из исходной коллекции, с каждой из них взято по три случайных кропа). Аугментация не проводилась.

RGB изображения процессились с помощью `keras.applications.resnet50.preprocess_input`. После данного препроцессинга, значения пикселей изображения лежат в пределах примерно от -123 до 150. Глубины подавались в модель в исходном виде в метрах.

### Модель

Архитектура модели взята из статьи: https://arxiv.org/pdf/1606.00373.pdf. Основу модели составляет сверточный энкодинг из ResNet50, предобученной на ImageNet. После этого добавляются 5 Up-projection блоков, описанных в статье. Перед каждым блоком добавляется слой BatchNormalization, после каждого блока - дропаут с коэффициентом 0.5.

Модель обучается в 4 этапа, по несколько эпох на каждый: сначала обучаются только разверточные слои, затем разверточные и три последних ResNet-блока, затем разверточные и 6 последних ResNet-блоков, затем все слои.

Параметры обучения сохранены в файле `params.txt`.

## Создание модели

In [1]:
import tensorflow as tf
import keras
import numpy as np
import os
import h5py

Using TensorFlow backend.


In [2]:
assert tf.test.is_gpu_available()

In [3]:
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image

In [4]:
from keras.layers import *
from keras.models import Model, load_model
from keras.optimizers import Adam

In [5]:
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

In [6]:
def apply_upprojection(nn):
    n_filters = int(nn.shape[-1])
    batchnorm = BatchNormalization()(nn)
    upsampling = UpSampling2D()(batchnorm)
    conv = Conv2D(n_filters // 2, kernel_size=5, padding='same')(upsampling)
    relu = Activation('relu')(conv)
    conv_3 = Conv2D(n_filters // 2, kernel_size=3, padding='same')(relu)
    conv_proj = Conv2D(n_filters // 2, kernel_size=5, padding='same')(upsampling)
    add = Add()([conv_3, conv_proj])
    relu_2 = Activation('relu')(add)
    dropout = Dropout(0.5)(relu_2)
    return dropout


def create_model():
    model = ResNet50(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
    for layer in model.layers:
        layer.trainable = False
    model.compile(loss='mean_squared_error', optimizer='adam')
    resnet_output = model.output
    nn = Conv2D(1024, kernel_size=1, padding='same')(resnet_output)
    for k in range(5):
        nn = apply_upprojection(nn)
    depth_output = Conv2D(1, kernel_size=3, padding='same')(nn)
    depth_output = Reshape((224, 224))(depth_output)
    fcrn_model = Model(inputs=model.input, outputs=depth_output)
    return fcrn_model

In [7]:
fcrn_model = create_model()
fcrn_model.summary()

/home/kmouraviev/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [8]:
fcrn_model.save('./model_pretrained.hdf5')

## Загрузка датасета

In [9]:
!echo $HDF5_USE_FILE_LOCKING

FALSE


In [10]:
try:
    data_file.close()
except:
    pass
data_file = h5py.File('NYU_dataset_hdf5/data.hdf5', 'r')
rgbs = data_file['data']
depths = data_file['label']

In [11]:
rgbs_train = rgbs[:40000]
rgbs_val = rgbs[40000:]
depths_train = depths[:40000]
depths_val = depths[40000:]

In [12]:
rgbs_train = np.array(rgbs_train)
rgbs_val = np.array(rgbs_val)
depths_train = np.array(depths_train)
depths_val = np.array(depths_val)

In [ ]:
datagen = image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
datagen.fit(rgbs_train)

In [ ]:
rgbs_train = np.array([x for x in rgbs_train])
depths_train = np.array([x[np.newaxis, :, :] for x in depths_train])

In [ ]:
train_flow = datagen.flow(rgbs_train, depths_train, batch_size=32)
validation_flow = datagen.flow(rgbs_val, depths_val, batch_size=32)

In [ ]:
fcrn_model.compile(optimizer=Adam(lr=1e-4, decay=1e-4), loss='mean_squared_error')

In [ ]:
fcrn_model.fit_generator(train_flow,
                         steps_per_epoch=10,
                         epochs=1,
                         validation_data=[rgbs_val, depths_val])

## Обучение

In [13]:
def train_model(fcrn_model, params):
    # initialize params
    batch_sizes = params['batch_size']
    lrs = params['lr']
    decays = params['decay']
    n_epochs = params['n_epoch']
    dir_to_save_weights = params['weights_dir']
    fcrn_model = load_model('./model_pretrained.hdf5')
    if not os.path.exists(dir_to_save_weights):
        os.mkdir(dir_to_save_weights)
    
    # Train network. First stage: only FCN layers
    fcrn_model.compile(optimizer=Adam(lr=lrs[0], decay=decays[0]),
                       loss='mean_squared_error'
                      )
    fcrn_model.fit(rgbs_train,
                   depths_train, 
                   batch_size=batch_sizes[0], 
                   epochs=n_epochs[0], 
                   validation_data=[rgbs_val, depths_val])
    fcrn_model.save(os.path.join(dir_to_save_weights, 'only_fcn_trained.hdf5'))
    
    # Train network. Second stage: FCN layers and three last ResNet blocks
    after_add12 = False
    for layer in fcrn_model.layers:
        layer.trainable = after_add12
        if layer.name == 'add_12':
            after_add12 = True
    fcrn_model.compile(optimizer=Adam(lr=lrs[1], decay=decays[1]),
                       loss='mean_squared_error'
                      )
    fcrn_model.fit(rgbs_train,
                   depths_train, 
                   batch_size=batch_sizes[1], 
                   epochs=n_epochs[1], 
                   validation_data=[rgbs_val, depths_val])
    fcrn_model.save(os.path.join(dir_to_save_weights, 'trained_since_add12.hdf5'))
    
    # Train network. Third stage: FCN layers and six last ResNet blocks
    after_add9 = False
    for layer in fcrn_model.layers:
        layer.trainable = after_add9
        if layer.name == 'add_9':
            after_add9 = True
    fcrn_model.compile(optimizer=Adam(lr=lrs[2], decay=decays[2]),
                       loss='mean_squared_error'
                      )
    fcrn_model.compile(optimizer=Adam(lr=lrs[2], decay=decays[2]),
                       loss='mean_squared_error'
                      )
    fcrn_model.fit(rgbs_train,
                   depths_train, 
                   batch_size=batch_sizes[2], 
                   epochs=n_epochs[2], 
                   validation_data=[rgbs_val, depths_val])
    fcrn_model.save(os.path.join(dir_to_save_weights, 'trained_since_add9.hdf5'))
    
    # Train network. Last stage: all layers are trainable
    for layer in fcrn_model.layers:
        layer.trainable = True
    fcrn_model.compile(optimizer=Adam(lr=lrs[3], decay=decays[3]),
                       loss='mean_squared_error'
                      )
    fcrn_model.fit(rgbs_train,
                   depths_train, 
                   batch_size=batch_sizes[3], 
                   epochs=n_epochs[3], 
                   validation_data=[rgbs_val, depths_val])
    fcrn_model.save(os.path.join(dir_to_save_weights, 'all_layers_trained.hdf5'))
    
    # Save parameters into directory with model weights
    with open(os.path.join(dir_to_save_weights, 'params.txt'), 'w') as f:
        f.write(str(params))

### Lot of epochs

In [ ]:
params = {
    'batch_size': (32, 16, 16, 16),
    'lr': (1e-5, 1e-5, 3e-6, 1e-6),
    'decay': (1e-4, 1e-4, 1e-4, 1e-4),
    'n_epoch': (10, 10, 5, 5),
    'weights_dir': './experiment5_weights'
}

In [ ]:
train_model(fcrn_model, params)

Train on 40000 samples, validate on 16541 samples
Epoch 1/10
40000/40000 [==============================] - 634s 16ms/step - loss: 3.0539 - val_loss: 1.1289
Epoch 2/10
40000/40000 [==============================] - 628s 16ms/step - loss: 1.5527 - val_loss: 1.0456
Epoch 3/10
40000/40000 [==============================] - 631s 16ms/step - loss: 1.3171 - val_loss: 0.9358
Epoch 4/10
40000/40000 [==============================] - 628s 16ms/step - loss: 1.1582 - val_loss: 0.8914
Epoch 5/10
40000/40000 [==============================] - 628s 16ms/step - loss: 1.0425 - val_loss: 0.9057
Epoch 6/10
40000/40000 [==============================] - 630s 16ms/step - loss: 0.9610 - val_loss: 0.8330
Epoch 7/10
40000/40000 [==============================] - 629s 16ms/step - loss: 0.8870 - val_loss: 0.8606
Epoch 8/10
40000/40000 [==============================] - 628s 16ms/step - loss: 0.8306 - val_loss: 0.8244
Epoch 9/10
40000/40000 [==============================] - 628s 16ms/step - loss: 0.7809 - val_